In [1]:
import numpy as np
import bz2
import pandas as pd
import pickle
np.random.seed(10)

In [2]:
RF_PATH="adult_randfor.bz2"
DATA_PATH="adult_randfor.data.npz"

Carico la random forest

In [3]:
with bz2.BZ2File(RF_PATH) as f:
    black_box = pickle.load(f)

/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
black_box

RandomForestClassifier(criterion='entropy', max_depth=350, max_features='sqrt',
                       min_samples_leaf=10, min_samples_split=10,
                       n_estimators=350)

In [5]:
loaded = np.load(DATA_PATH, allow_pickle=True)
for key in loaded.keys():
    print(key)

x_train
x_test
y_train
y_test
x_test_clustered
y_test_clustered


x_test_clustered contiene i 5 elementi più vicini ad ogni centroide, estratti dal test set.

In [6]:
x_train, x_test, x_test_clustered = loaded['x_train'], loaded['x_test'], loaded['x_test_clustered']
y_train, y_test, y_test_clustered = loaded['y_train'], loaded['y_test'], loaded['y_test_clustered']

Nei file csv sono contenute anche le informazioni relative ai cluster 

In [7]:
trainset = pd.read_csv("adult_trainset.csv")
testset  = pd.read_csv("adult_testset.csv")
closest = pd.read_csv("adult_clustered_5_closest.csv")

La prima colonna, `Unnamed: 0` contiene l'indice della riga nel dataset adult originale, deve essere scartata, era lì solo per un eventuale debug. (closest ne ha due da scartare)

In [8]:
trainset.head(1)

,Unnamed: 0,Age,Workclass,Fnlwgt,Education,Education-num,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,Target,Cluster
0,7662,0.11867,0.285829,-0.895058,0.421658,1.128996,0.048399,0.485342,0.106689,0.263855,0.313955,-0.147502,-0.218673,0.756794,0.254411,0,1


In [9]:
closest.head(1)

,Unnamed: 0,Unnamed: 0.1,Age,Workclass,Fnlwgt,Education,Education-num,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,Target,Cluster
0,2642,10006,-0.719029,0.218918,-0.298864,0.164328,-0.440434,0.048399,0.116515,0.066334,0.263855,0.313955,-0.147502,-0.218673,-0.495444,0.254411,0,0


Separo features / target / cluster

In [10]:
trainset_features  = trainset.iloc[:,1:-2]
trainset_targets   = trainset['Target']
trainset_cluster   = trainset['Cluster']

testset_features   = testset.iloc[:,1:-2]
testset_targets    = testset['Target']
testset_cluster    = testset['Cluster']

closest_features = closest.iloc[:,2:-2]
closest_targets  = closest['Target']
closest_cluster  = closest['Cluster']

Controllo che i dati nei csv siano effettivamente quelli usati per allenare la random forest.

In [11]:
assert (trainset_features.to_numpy() == x_train).all().all()
assert (testset_features.to_numpy() == x_test).all().all()
assert (closest_features.to_numpy() == x_test_clustered).all().all()
assert (trainset_targets.to_numpy() == y_train).all().all()
assert (testset_targets.to_numpy() == y_test).all().all()
assert (closest_targets.to_numpy() == y_test_clustered).all().all()